<a href="https://colab.research.google.com/github/philosophynote/machine_learning/blob/main/prtimes_rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import re
title = "調達"
if re.search("調達", title):
  print("ttt")
# "調達".in(title)
# title.find("調達")

ttt


In [62]:
import json
from datetime import datetime,timezone,timedelta
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import feedparser
import pytz
import re
from time import mktime

RSS_URL = "https://prtimes.jp/index.rdf"
SLACK_CHANNEL = ""
SLACK_USERNAME = ""
SLACK_URL = ""
THRESHOLD = 8

def post_slack(text):
    send_data = {
        "channel": SLACK_CHANNEL,
        "username": SLACK_USERNAME,
        "text": text
    }
    payload = "payload=" + json.dumps(send_data)
    request = Request(
        SLACK_URL,
        data=payload.encode("utf-8"),
        method="POST"
        )
    with urlopen(request) as response:
        response_body = response.read().decode("utf-8")

def is_financing(title, keyword_text):
    return re.search("調達", title) or re.search("調達", keyword_text)

def lambda_handler():
    try:
        # 対象のRSSフィードを取得
        feed = feedparser.parse(RSS_URL)
        target_list = []

        for entry in feed.entries:
          now = datetime.now(pytz.timezone('Asia/Tokyo'))
          entry_date = datetime.fromtimestamp(mktime(entry.updated_parsed), pytz.timezone('Asia/Tokyo'))
          threshold = now - timedelta(hours=THRESHOLD)

          if threshold <= entry_date < now:
            url = entry.link
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            keyword = soup.find('dl', {'class': 'entry-info-renew'})
            if keyword and is_financing(entry.title, keyword.text):
              tmp = entry.title + '\n' + entry.link + '\n'
              target_list.append(tmp)

        if len(target_list) > 0:
            # 更新されていればSlackに投稿
            print('post slack')
            post_slack('\n'.join(target_list))

    except Exception as e:
        print(e)
        raise e
      

In [63]:
lambda_handler()

post slack


In [65]:
now = datetime.now(pytz.timezone('Asia/Tokyo'))
entry_date = datetime.fromtimestamp(mktime(tmp.updated_parsed), pytz.timezone('Asia/Tokyo'))
threshold = now - timedelta(hours=THRESHOLD)

In [64]:
tmp = feed['entries'][0]
tmp

{'id': 'https://prtimes.jp/main/html/rd/p/000000002.000110150.html',
 'title': '躍動と色彩のドラマー羽生一子率いるHABUBAN 新作CD「Time for the Stars」3月30日リリース',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://prtimes.jp/index.rdf',
  'value': '躍動と色彩のドラマー羽生一子率いるHABUBAN 新作CD「Time for the Stars」3月30日リリース'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://prtimes.jp/main/html/rd/p/000000002.000110150.html'}],
 'link': 'https://prtimes.jp/main/html/rd/p/000000002.000110150.html',
 'summary': '[ミラレソ株式会社]\n[画像1: https://prtimes.jp/i/110150/2/resize/d110150-2-26a4c48a9b7c2bae3230-0.jpg ]\n\nベーシスト、プロデューサー 飯田雅春による新レーベル Milareso Records(ミラレソレコード)第2弾。\n\n【HABUBAN】\n羽生...',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://prtimes.jp/index.rdf',
  'value': '[ミラレソ株式会社]\n[画像1: https://prtimes.jp/i/110150/2/resize/d110150-2-26a4c48a9b7c2bae3230-0.jpg ]\n\nベーシスト、プロデューサー 飯田雅春による新レーベル Milareso Records(ミラレソレコード)第2弾。\n\n【HABUBAN】\n羽生...'}

In [ ]:


now = datetime.now(pytz.timezone('Asia/Tokyo'))
entry_date = datetime.fromtimestamp(mktime(tmp.updated_parsed), pytz.timezone('Asia/Tokyo'))
threshold = now - timedelta(hours=THRESHOLD)


In [66]:
now

datetime.datetime(2023, 2, 26, 21, 21, 17, 604580, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)

In [67]:
entry_date

datetime.datetime(2023, 2, 26, 16, 40, 2, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)

In [68]:
threshold

datetime.datetime(2023, 2, 26, 13, 21, 17, 604580, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)

In [ ]:
if threshold <= entry_date < now:
  url = tmp.link
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  keyword = soup.find('dl', {'class': 'entry-info-renew'})
  # if keyword.text.find("資金調達") != -1:
  #     continue
  tmp = tmp.title + '\n' + tmp.link + '\n' + tmp.updated + '\n' 
  print(tmp)

In [ ]:
threshold <= entry_date < now

False